In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install datasets
!pip install evaluate

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.34.1
Uninstalling transformers-4.34.1:
  Successfully uninstalled transformers-4.34.1
Found existing installation: accelerate 0.24.0
Uninstalling accelerate-0.24.0:
  Successfully uninstalled accelerate-0.24.0
  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
  Using cached accelerate-0.24.0-py3-none-any.whl (260 kB)


In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

#nltk.download("punkt")

In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/kunalt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [14]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail","2.0.0")

# Access the training, validation, and test splits
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

In [9]:
train_data

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [10]:
# Access the first example in the training split
example = train_data[0]
source_text = example["article"]
target_text = example["highlights"]

# Print the source text and target summary
print("Source Text:")
print(source_text)
print("\nTarget Summary:")
print(target_text)

Source Text:
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. D

In [11]:
split_lengths = [len(dataset[split])for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\narticle:")

print(dataset["test"][1]["article"])

print("\nSummary:")

print(dataset["test"][1]["highlights"])

Split lengths: [287113, 13368, 11490]
Features: ['article', 'highlights', 'id']

article:
(CNN)Never mind cats having nine lives. A stray pooch in Washington State has used up at least three of her own after being hit by a car, apparently whacked on the head with a hammer in a misguided mercy killing and then buried in a field -- only to survive. That's according to Washington State University, where the dog -- a friendly white-and-black bully breed mix now named Theia -- has been receiving care at the Veterinary Teaching Hospital. Four days after her apparent death, the dog managed to stagger to a nearby farm, dirt-covered and emaciated, where she was found by a worker who took her to a vet for help. She was taken in by Moses Lake, Washington, resident Sara Mellado. "Considering everything that she's been through, she's incredibly gentle and loving," Mellado said, according to WSU News. "She's a true miracle dog and she deserves a good life." Theia is only one year old but the dog's b

In [12]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['highlights'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [15]:
dataset_pt = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/home/kunalt/Tools/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [16]:
dataset_pt["train"]

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})

In [17]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [18]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='/home/kunalt/Text/pegasus-cnndailymail', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [19]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["test"],
                  eval_dataset=dataset_pt["validation"])

In [20]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.588600,1.429903


TrainOutput(global_step=718, training_loss=1.6934242859524273, metrics={'train_runtime': 32136.8977, 'train_samples_per_second': 0.358, 'train_steps_per_second': 0.022, 'total_flos': 2.370145546265395e+16, 'train_loss': 1.6934242859524273, 'epoch': 1.0})

In [21]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [24]:
import evaluate

In [25]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load('rouge')

In [26]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'article', column_summary= 'highlights'
)

100%|███████████████████████████████████████████████████████████████████| 5/5 [01:01<00:00, 12.31s/it]


AttributeError: 'numpy.float64' object has no attribute 'mid'

In [36]:
score

{'rouge1': 0.02015273712174629,
 'rouge2': 0.002387620781403165,
 'rougeL': 0.020328111308326118,
 'rougeLsum': 0.0201563433024961}

In [27]:
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

AttributeError: 'numpy.float64' object has no attribute 'mid'

In [37]:
## Save model
model_pegasus.save_pretrained("/home/kunalt/Text/pegasus-cnn-model")

In [38]:
## Save tokenizer
tokenizer.save_pretrained("/home/kunalt/Text/tokenizer")

('/home/kunalt/Text/tokenizer/tokenizer_config.json',
 '/home/kunalt/Text/tokenizer/special_tokens_map.json',
 '/home/kunalt/Text/tokenizer/spiece.model',
 '/home/kunalt/Text/tokenizer/added_tokens.json',
 '/home/kunalt/Text/tokenizer/tokenizer.json')

In [39]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/home/kunalt/Text/tokenizer")

In [40]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["article"]

reference = dataset["test"][0]["highlights"]

pipe = pipeline("summarization", model="pegasus-cnn-model",tokenizer=tokenizer)

##
print("Article:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Article:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's cer

In [41]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][5]["article"]

reference = dataset["test"][5]["highlights"]

pipe = pipeline("summarization", model="pegasus-cnn-model",tokenizer=tokenizer)

##
print("Article:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Article:
(CNN)He's a blue chip college basketball recruit. She's a high school freshman with Down syndrome. At first glance Trey Moses and Ellie Meredith couldn't be more different. But all that changed Thursday when Trey asked Ellie to be his prom date. Trey -- a star on Eastern High School's basketball team in Louisville, Kentucky, who's headed to play college ball next year at Ball State -- was originally going to take his girlfriend to Eastern's prom. So why is he taking Ellie instead? "She's great... she listens and she's easy to talk to" he said. Trey made the prom-posal (yes, that's what they are calling invites to prom these days) in the gym during Ellie's P.E. class. Trina Helson, a teacher at Eastern, alerted the school's newspaper staff to the prom-posal and posted photos of Trey and Ellie on Twitter that have gone viral. She wasn't surpristed by Trey's actions. "That's the kind of person Trey is," she said. To help make sure she said yes, Trey entered the gym armed with flo

In [42]:
# summmarization of news articwel
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = '''Afghanistan's thumping victory over Pakistan in the ongoing World Cup tournament has created ripples across the world with people now acknowledging the progress made by the war-torn country. Not just Pakistan, the Afghans also defeated the former world champion England in their earlier encounter which goes on to show that it's not just an one-off upset but a concreted effort by a team that has really worked hard on their game.
Reacting to this famous win against Pakistan, Taliban lavished huge praise on the Afghanistan cricket team. Khalid Zardan, a Kabul Police spokesperson who is also connected with Taliban said, "Our cricket team has achieved a historic win. We won a match which many believed was impossible for us. This victory is a message to the people that we are progressing. Watch out for us but do not trouble us."The Chief of Staff of the Prime Minister's Office of the Taliban government in Afghanistan has also congratulated the Afghanistan cricket team on their Twitter account.

The statement posted on the account reads, "The Afghanistan national cricket team has defeated Pakistan. Congratulations to the national cricket team, cricket board and all Afghan citizens on this victory."

Following Afghanistan's historic win against Pakistan, people came out in large numbers and also played music despite an effective ban on song and dance dictated by the Taliban government.
About this epic encounter, Afghanistan produced a superb batting and bowling display to chase down a target of 283 set by Pakistan, their highest successful chase in ODI cricket. It was also their first-ever victory over Pakistan in eight ODIs, and came just a week after their shock victory over defending champions England.

Afghanistan's bowlers restricted Pakistan to 282 for seven in 50 overs, before their batsmen chased down the target with eight wickets and an over to spare. Ibrahim Zadran top-scored for Afghanistan with 87, while Rahmanullah Gurbaz scored 65 and Rahmat Shah scored 55.'''

#reference = dataset["test"][5]["highlights"]

pipe = pipeline("summarization", model="pegasus-cnn-model",tokenizer=tokenizer)

##
print("Article:")
print(sample_text)


#print("\nReference Summary:")
#print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

################################



Article:
Afghanistan's thumping victory over Pakistan in the ongoing World Cup tournament has created ripples across the world with people now acknowledging the progress made by the war-torn country. Not just Pakistan, the Afghans also defeated the former world champion England in their earlier encounter which goes on to show that it's not just an one-off upset but a concreted effort by a team that has really worked hard on their game.
Reacting to this famous win against Pakistan, Taliban lavished huge praise on the Afghanistan cricket team. Khalid Zardan, a Kabul Police spokesperson who is also connected with Taliban said, "Our cricket team has achieved a historic win. We won a match which many believed was impossible for us. This victory is a message to the people that we are progressing. Watch out for us but do not trouble us."The Chief of Staff of the Prime Minister's Office of the Taliban government in Afghanistan has also congratulated the Afghanistan cricket team on their Twitte